In [ ]:
!pip3 install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.0 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import kerastuner as kt

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
def create_model(hp):
  num_hidden_layers = 1
  num_units = 8
  dropout_rate = 0.1
  learning_rate = 0.01

  if hp:
    num_hidden_layers = hp.Choice('num_hidden_layers', values=[1,2,3])
    num_units = hp.Choice('num_units', values=[8, 16, 32])
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5)
    learning_rate = hp.Float('learning_rate', min_value=0.0001, max_value=0.01)

  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
  model.add(tf.keras.layers.Lambda(lambda x: x/255.))

  for _ in range(0, num_hidden_layers):
    model.add(tf.keras.layers.Dense(num_units, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout_rate))

  model.add(tf.keras.layers.Dense(10, activation='softmax'))
  model.compile(loss='sparse_categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])


  return model


In [ ]:
create_model(None).summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 lambda_1 (Lambda)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 8)                 6280      
                                                                 
 dropout_3 (Dropout)         (None, 8)                 0         
                                                                 
 dense_5 (Dense)             (None, 10)                90        
                                                                 
Total params: 6370 (24.88 KB)
Trainable params: 6370 (24.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:

class CustomTuner(kt.tuners.BayesianOptimization):
  def run_trial(self, trial, *args, **kwargs):
    kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 32, 128, step=32)
    return super(CustomTuner, self).run_trial(trial, *args, **kwargs)





In [ ]:

tuner = CustomTuner(create_model, objective='val_accuracy', max_trials=20, directory='logs', project_name='fashion_mnist', overwrite=True)



In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_hidden_layers (Choice)
{'default': 1, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}
num_units (Choice)
{'default': 8, 'conditions': [], 'values': [8, 16, 32], 'ordered': True}
dropout_rate (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': None, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'linear'}


In [ ]:
tuner.search(x_train, y_train, validation_data=(x_test, y_test), epochs=5, verbose=1)

Trial 20 Complete [00h 00m 34s]
val_accuracy: 0.8345000147819519

Best val_accuracy So Far: 0.8546000123023987
Total elapsed time: 00h 06m 52s


In [ ]:
tuner.get_best_hyperparameters()


In [ ]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
for hp in best_hp.space:
  print(f"{hp.name}: {best_hp.get(hp.name)}")

num_hidden_layers: 1
num_units: 32
dropout_rate: 0.1
learning_rate: 0.002986268006329165
batch_size: 128


In [ ]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 lambda (Lambda)             (None, 784)               0         
                                                                 
 dense (Dense)               (None, 32)                25120     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                330       
                                                                 
Total params: 25450 (99.41 KB)
Trainable params: 25450 (99.41 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:

final_model = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=128, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)])


Epoch 1/20
469/469 [==============================] - 4s 6ms/step - loss: 0.3820 - accuracy: 0.8615 - val_loss: 0.3903 - val_accuracy: 0.8610
Epoch 2/20
469/469 [==============================] - 2s 4ms/step - loss: 0.3707 - accuracy: 0.8650 - val_loss: 0.3885 - val_accuracy: 0.8624
Epoch 3/20
469/469 [==============================] - 2s 4ms/step - loss: 0.3654 - accuracy: 0.8666 - val_loss: 0.3868 - val_accuracy: 0.8645
Epoch 4/20
469/469 [==============================] - 2s 4ms/step - loss: 0.3596 - accuracy: 0.8672 - val_loss: 0.3843 - val_accuracy: 0.8637
Epoch 5/20
469/469 [==============================] - 2s 3ms/step - loss: 0.3541 - accuracy: 0.8685 - val_loss: 0.3811 - val_accuracy: 0.8666
Epoch 6/20
469/469 [==============================] - 2s 4ms/step - loss: 0.3489 - accuracy: 0.8711 - val_loss: 0.3764 - val_accuracy: 0.8659
Epoch 7/20
469/469 [==============================] - 3s 6ms/step - loss: 0.3500 - accuracy: 0.8707 - val_loss: 0.4069 - val_accuracy: 0.8538
Epoch 